In [15]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [16]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-05-09 23:54:42.412616


C:\Users\HP\AppData\Local\Temp\ipykernel_14912\2672588006.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_date = pd.to_datetime(datetime.utcnow())


In [17]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [18]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [19]:
current_date.to_datetime64()

numpy.datetime64('2025-05-09T23:00:00.000000000')

In [20]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [21]:
fetch_data_to

Timestamp('2025-05-09 23:00:00+0000', tz='UTC')

In [22]:
fetch_data_from

Timestamp('2025-04-10 23:00:00+0000', tz='UTC')

In [25]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_bike_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_bike_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [26]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

JC-202404-citibike-tripdata.csv
Successfully downloaded data for 2024-04.
Loading data for 2024-04...


C:\Users\HP\Downloads\sp25_taxi-main\src\data_utils.py:93: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  location_filter = ~rides["start_station_id"].astype(str).str.contains(r'^\d+(\.\d+)?$', regex=True)


Total records: 79,116
Valid records: 79,071
Records dropped: 45 (0.06%)
Successfully processed data for 2024-04.
Combining all monthly data...
Data loading and processing complete!


TypeError: Invalid comparison between dtype=datetime64[ns] and Timestamp

In [12]:
rides

,pickup_datetime,pickup_location_id
5359,2025-01-09 04:51:34,3
40021,2025-01-09 13:09:55,3
107013,2025-01-10 04:38:43,3
107628,2025-01-10 05:34:12,3
108750,2025-01-10 06:37:19,3
...,...,...
2820212,2025-02-06 22:57:08,263
2817070,2025-02-06 22:57:10,263
2821718,2025-02-06 22:57:37,263
2822944,2025-02-06 22:57:47,263


In [14]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [15]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-01-08 23:00:00,3,0
1,2025-01-09 00:00:00,3,0
2,2025-01-09 01:00:00,3,0
3,2025-01-09 02:00:00,3,0
4,2025-01-09 03:00:00,3,0
...,...,...,...
176779,2025-02-06 18:00:00,263,120
176780,2025-02-06 19:00:00,263,100
176781,2025-02-06 20:00:00,263,114
176782,2025-02-06 21:00:00,263,124


In [16]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176784 entries, 0 to 176783
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         176784 non-null  datetime64[ns]
 1   pickup_location_id  176784 non-null  int16         
 2   rides               176784 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [18]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-02-06 19:04:02,424 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-06 19:04:02,435 INFO: Initializing external client
2025-02-06 19:04:02,436 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-06 19:04:03,525 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551


In [19]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |███████████████████████| Rows 176784/176784 | Elapsed Time: 00:43 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1211551/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)